# libs, dataset

In [ ]:
!rm -rf used_cars

In [ ]:
!git lfs clone https://huggingface.co/datasets/imbalet/used_cars

In [ ]:
!pip install numpy pandas scikit-learn tensorflow

# import

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

# Creating db
in progress...

In [ ]:
vars = ["mark", "model", "super_gen"]
v = ["engine", "transmission"]

In [ ]:
for i in set(df["model"]):
    if len(set(df[df["model"] == i]["mark"])) > 1: print(i, set(df[df["model"] == i]["mark"]))

In [ ]:
data = {}
for index, row in df.iterrows():
    mark = row["mark"]
    model = row["model"]
    gen = row["super_gen"]
    tr = row["transmission"]
    en = row["engine"]
    if mark not in data:
        data[mark] = {}
    if model not in data[mark]:
        data[mark][model] = {"super_gen": set(), "transmission": set(), "engine": set(), "year": set()}
    data[mark][model]["super_gen"].add(str(gen))
    data[mark][model]["transmission"].add(str(tr))
    data[mark][model]["engine"].add(str(en))
    data[mark][model]["year"].add(row["year"])



In [ ]:
for i in data:
    for j in data[i]:
        data[i][j]["super_gen"] = list(data[i][j]["super_gen"])
        data[i][j]["transmission"] = list(data[i][j]["transmission"])
        data[i][j]["engine"] = list(data[i][j]["engine"])
        data[i][j]["year"] = list(data[i][j]["year"])

        print(data[i][j])

In [ ]:
cols = ["owners", 'region', 'body_type', 'steering_wheel', 'gear_type', "color"]
vars = {i: list(map(str, set(df[i]))) for i in cols}

In [ ]:
vars["mark"] = data

In [ ]:
import json
with open("variants_cars.json", 'w') as f:
    json.dump(data, f, ensure_ascii = False)
    

In [ ]:
with open("variants.json", 'w') as f:
    json.dump(vars, f, ensure_ascii = False)

# dataset


In [ ]:
df = pd.read_csv("used_cars/dataset.csv")

In [ ]:
df.sample(3)

In [ ]:
df = df.drop(["doors", "class"], axis = 1)

In [ ]:
# df["doors"] = df["doors"].astype("object")
df["super_gen"] = df["super_gen"].astype("object")
df["year"] = df["year"].astype("object")
df["owners"] = df["owners"].astype("object")

In [ ]:
num_features = ["owners", "mileage", "power", "displacemnt"]
cat_features = df.select_dtypes(include=['object']).columns

In [ ]:
price = ["price"]

price_scaler = StandardScaler()

df[price] = price_scaler.fit_transform(df[price])

In [ ]:
scaler = StandardScaler()
df[num_features] = scaler.fit_transform(df[num_features])

In [ ]:
label_encoders = {}
for feature in cat_features:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    label_encoders[feature] = le


In [ ]:
# creating config scale/encoders file
import json
scalers = {scaler.feature_names_in_[i]: (float(scaler.mean_[i]), float(scaler.scale_[i])) for i in range(len(scaler.mean_))}
scalers["price"] = (float(price_scaler.mean_[0]), float(price_scaler.scale_[0]))

labels = {i: list(map(str, label_encoders[i].classes_)) for i in label_encoders}

with open("configs.json", 'w', encoding="utf-8") as f:
    json.dump({"scalers": scalers, "labels": labels}, f, ensure_ascii=False, indent=4)

In [ ]:
df.sample(3)

In [ ]:
X = df.drop(columns=['price'])
y = df['price']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# model


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

In [ ]:
model.compile(optimizer='adam', loss = "mean_squared_error")

In [ ]:
model.fit(X_train, y_train, epochs=50, validation_split=0.1, batch_size=64)

In [ ]:
model.save('model.keras')

In [ ]:
test_loss = model.evaluate(X_test, y_test)
print(f'Test loss (MSE): {test_loss}')

In [ ]:
original_data = price_scaler.inverse_transform(np.array(y_test).reshape(-1, 1))

In [ ]:
pred_unscaled = price_scaler.inverse_transform(model.predict(X_test))

In [ ]:
# MAE
sum(abs(original_data - pred_unscaled)) / original_data.shape[0]